To create a bike count table:
 - Load WSDOT count data in text format into a new geodatabase (gdb)
 - Load copy of TransRefEdges into same gdb
 - Convert WSDOT data into points file by locating X and Y cooridnates of each collection location
     - Use a geographic coordinate system of WGS 1984
 - Export that points file to the gdb
 - intersect points file with geodb, using a tolerance of ~ 50 feet, since points do not line up   perfectly with TransRefEdges
 - Export resulting intersection table as counts_intersect.csv

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv(r'R:\Bike\Counts\counts_intersect.csv')

In [9]:
original_df = len(df)

The intersect joined multiple edges to each point because there are separate edges for non-motorized and vehicle streets. We should set a heirarchy so that bike counts represent the non-motorized links first, if possible. 



In [10]:
# First way to filter is to remove and links that don't allow either k, l, or q modes (the 3 different bike modes)

results = []

for row in xrange(len(df)):
    if any(x in df['Modes'].iloc[row] for x in ['k','l','q']):
        results.append(df.iloc[row].values)
        
df = pd.DataFrame(results, columns = df.columns)
print str(original_df - len(df)) + ' rows removed'

531 rows removed


Ideally we want to separate bike trails from regular links so we are sure to be assignming trips along bike links
However, for now we can just focus on getting the total link volumes and concentrating on link types later

In [11]:
# collect all EdgeIDs and total volumes
df[['OBJECTID', 'PSRCEdgeID','INode','JNode','Bike_Total','CountYear','CountTime','CountDate']]

# Since assume the count location includes the nearby links (within 50 ft of reported location),
# we want a list of all links to pass into Emme to sum up counts.
# For comparison, we need to group each count by OBJECTID so we have a single value for each count location/time.
# Note that some locations have multiple counts!

df[['OBJECTID', 'PSRCEdgeID','INode','JNode']].to_csv('bike_count_links.csv')

In [12]:
# Get unique list of counts
count_totals = pd.DataFrame(df.groupby('OBJECTID').mean()['Bike_Total'])
# Note we are aggregating with "mean" but all bike_total values are the same for 
# rows with the same OBJECTID

In [13]:
count_totals['OBJECTID'] = count_totals.index
count_totals.to_csv('bike_count_values.csv')

In [14]:
## Pass in the bike_count_links.csv to the bike_model to get the assigned values back
# Then read them in to compare to the bike_count_values

In [15]:
df_counts = pd.read_csv(r'D:\soundcast\soundcast\scripts\summarize\notebooks\bike_count_links.csv')

In [19]:
# Edges shapefile needs to be exported as csv, could do this with arcpy, as part of 
# I-O tool?
# For now, I just exported a 2010 network manually
edges_df = pd.read_csv(r'R:\Bike\Counts\edges_0.txt')

In [20]:
edges_df

,FID,OBJECTID,Enabled,PSRCEdgeID,FTRsegID,FacilityTy,NewFacilit,INode,JNode,InServiceD,...,SplitTR,SplitTK,Updated1,prjRte,shptype,Dissolve,Direction,Shape_len,NewINode,NewJNode
0,0,89885983,0,4649,223160,6,3,144702,144696,2000,...,,,,,,0,0,339.858275,148702,148696
1,1,89885986,0,3193,0,6,3,115584,115589,2000,...,,,,,,0,0,1518.841019,119584,119589
2,2,89885987,0,3201,0,6,3,115589,115598,2000,...,,,,,,0,0,1105.932993,119589,119598
3,3,89885988,0,3202,125055,6,3,115598,115606,2000,...,,,,,,0,0,1320.687178,119598,119606
4,4,89885990,0,3864,161061,7,3,149482,149511,1995,...,,,,,,0,0,379.998661,153482,153511
5,5,89885993,0,595,128628,7,6,119525,119529,1995,...,,,,,,0,0,287.065194,123525,123529
6,6,89886003,0,2359,222309,1,1,67301,67533,2000,...,,,,,,0,1,321.567210,71301,71533
7,7,89886004,0,2360,222309,1,1,67533,67639,2000,...,,,,,,0,1,163.516684,71533,71639
8,8,89886012,0,5530,68390,3,2,64363,64348,2000,...,,,,,,0,0,39.593654,68363,68348
9,9,89886015,0,5574,70408,5,3,65863,65913,2000,...,,,,,,0,0,97.461642,69863,69913


In [37]:
df = df_counts.merge(edges_df, on=['INode','JNode'])
df.index = df[['INode','JNode']]

In [39]:
# list_model_vols = []

# tod = bike_assignment_tod

# for item in df.index:
#     i = list(item)[0]
#     j = list(item)[1]
#     link = network.link(i, j)
#     x = {}
#     x['INode'] = i
#     x['JNode'] = j
#     if link != None:
#         x['vol' + tod] = link['@bvol']
#     else:
#         x['vol' + tod] = None
#     list_model_vols.append(x)
# print len(list_model_vols)
# df =  pd.DataFrame(list_model_vols)
# # df = df.set_index(['NewINode', 'NewJNode'])
# # return df
# df.to_csv('bike_count_output.csv')

In [41]:
# Added above to bike_model.py, returned the bike_count_output.csv, which we open below
emme_results = pd.read_csv(r'D:/soundcast/soundcast/bike_count_output.csv')

In [47]:
emme_results.merge(df_counts, left_on=['INode','JNode'], right_on=['INode','JNode'])

,Unnamed: 0_x,INode,JNode,vol7to8,Unnamed: 0_y,OBJECTID,PSRCEdgeID


In [48]:
df_counts

,Unnamed: 0,OBJECTID,PSRCEdgeID,INode,JNode
0,0,1354,121177,171057,171089
1,1,1354,122235,171089,171289
2,2,1354,122243,171462,171057
3,3,1354,122249,171057,170869
4,4,1354,247872,170879,171089
5,5,1354,1364,171089,171620
6,6,1355,121177,171057,171089
7,7,1355,122235,171089,171289
8,8,1355,122243,171462,171057
9,9,1355,122249,171057,170869
